In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter


def str_to_list(string):
    str_list = string.str.split('|')
    return str_list


data = pd.read_csv('movie_bd_v5.csv')

data['profit'] = (data.revenue - data.budget)
data['gen_list'] = str_to_list(data['genres'])




In [2]:
#самый кассовый режисер

data['dir_list'] = str_to_list(data['director'])
data_dir = data.explode('dir_list')
(display(data_dir.groupby(['dir_list'])
         ['revenue'].sum().sort_values(ascending=False).head(1)))

dir_list
Peter Jackson    6490593685
Name: revenue, dtype: int64

In [3]:
#режиссер,снявший больше всего фильмов в стиле Action
data_action = data[data.genres.str.contains('Action', na=False)]
Counter(data_action.explode('dir_list').dir_list).most_common(1)

[('Robert Rodriguez', 9)]

In [4]:
#режиссер,снявший больше всего фильмов в стиле Action 2
(Counter(data[data.genres.str.contains('Action', na=False)]
         .explode('dir_list').dir_list).most_common(1))

[('Robert Rodriguez', 9)]

In [5]:
data_cast = data.copy()
data_cast['cast_list'] = str_to_list(data_cast['cast'])
data_cast = data_cast.explode('cast_list')

In [6]:
#Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году
data_cast_2012 = data_cast[data_cast.release_year == 2012]
(display(data_cast_2012.groupby(['cast_list'])
         ['revenue'].sum().sort_values(ascending=False).head(1)))

cast_list
Chris Hemsworth    2027450773
Name: revenue, dtype: int64

In [7]:
#Какой актер снялся в большем количестве высокобюджетных фильмов? 
#Примечание: в фильмах, где бюджет выше среднего по данной выборке.
#1
data_cast_budmean = data_cast[data_cast.budget>data_cast.budget.mean()]
Counter(data_cast_budmean.cast_list).most_common(1)

[('Matt Damon', 18)]

In [8]:
#Какой актер снялся в большем количестве высокобюджетных фильмов? 
#Примечание: в фильмах, где бюджет выше среднего по данной выборке.
#2
(Counter(data_cast[data_cast.budget>data_cast.budget.mean()]
         .cast_list).most_common(1))

[('Matt Damon', 18)]

In [9]:
#В фильмах какого жанра больше всего снимался Nicolas Cage?
#1
data_cast_Cage = data_cast[data_cast.cast_list == 'Nicolas Cage']
Counter(data_cast_Cage.explode('gen_list').gen_list).most_common()

[('Action', 17),
 ('Thriller', 15),
 ('Drama', 12),
 ('Crime', 10),
 ('Fantasy', 8),
 ('Adventure', 7),
 ('Comedy', 6),
 ('Science Fiction', 4),
 ('Mystery', 3),
 ('Animation', 3),
 ('Family', 3),
 ('History', 2),
 ('War', 1),
 ('Horror', 1),
 ('Romance', 1)]

In [10]:
#В фильмах какого жанра больше всего снимался Nicolas Cage?
#2
data_cast_Cage = data_cast[data_cast.cast_list == 'Nicolas Cage']
(Counter(data_cast[data_cast.cast_list == 'Nicolas Cage']
         .explode('gen_list').gen_list).most_common(1))

[('Action', 17)]

In [11]:
data_prod = data.copy()
data_prod['prod_list'] = str_to_list(data_prod['production_companies'])
data_prod = data_prod.explode('prod_list')

In [12]:
#Самый убыточный фильм от Paramount Pictures
data_prod_param = data_prod[data_prod.prod_list == 'Paramount Pictures']
(data_prod_param[data_prod_param
                 .profit == data_prod_param.profit.min()].original_title)

925    K-19: The Widowmaker
Name: original_title, dtype: object

In [13]:
#Какой год стал самым успешным по суммарным кассовым сборам?
(display(data.groupby(['release_year'])
         ['revenue'].sum().sort_values(ascending=False).head(1)))

release_year
2015    25449202382
Name: revenue, dtype: int64

In [14]:
#Какой самый прибыльный год для студии Warner Bros?
data_prod_warner = data_prod[data_prod.prod_list.str.contains('Warner Bros')]
(display(data_prod_warner.groupby(['release_year'])
         ['profit'].sum().sort_values(ascending=False).head()))

release_year
2014    2292949646
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
Name: profit, dtype: int64

In [15]:
#В каком месяце за все годы суммарно вышло больше всего фильмов?
data_month = data.copy()


def month(string):
    mth = string.str.split('/').str[0]
    return mth


data_month['month_list'] = month(data_month['release_date'])

Counter(data_month['month_list']).most_common(1)

[('9', 227)]

In [16]:
#Сколько суммарно вышло фильмов летом (за июнь, июль, август)
data_month_summer = data_month[data_month.month_list.isin(['6', 
                                                           '7', '8'])].copy()
data_month_summer.imdb_id.count()

450

In [17]:
#Для какого режиссера зима — самое продуктивное время года?
data_month_winter = data_month[data_month.month_list.isin(['12', 
                                                           '1', '2'])].copy()
data_month_winter = data_month_winter.explode('dir_list')
Counter(data_month_winter['dir_list']).most_common(5)

[('Peter Jackson', 7),
 ('Clint Eastwood', 6),
 ('Steven Soderbergh', 6),
 ('Shawn Levy', 4),
 ('Nancy Meyers', 4)]

In [18]:
# Какая студия даёт самые длинные названия своим фильмам по количеству символов?
data_prod['len_tittle'] = data_prod.original_title.apply(lambda x: len(x))
data_prod.groupby(['prod_list']
                 )['len_tittle'].mean().sort_values(ascending=False).head()

prod_list
Four By Two Productions      83.0
Jim Henson Company, The      59.0
Dos Corazones                47.0
Museum Canada Productions    46.0
Polsky Films                 46.0
Name: len_tittle, dtype: float64

In [19]:
#Описания фильмов какой студии в среднем самые длинные по количеству слов?
data_prod['len_overview'] = data_prod.overview.apply(lambda x: len(x.split(' ')))
data_prod.groupby(['prod_list']
                 )['len_overview'].mean().sort_values(ascending=False).head()

prod_list
Midnight Picture Show               175.0
Room 9 Entertainment                161.0
Heineken Branded Entertainment      161.0
98 MPH Productions                  159.0
Brookwell-McNamara Entertainment    156.0
Name: len_overview, dtype: float64

In [20]:
#Какие фильмы входят в один процент лучших по рейтингу?
data.sort_values(by='vote_average', ascending=False).head(int(len(data)*0.01))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,gen_list,dir_list
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825,"[Drama, Action, Crime, Thriller]",[Christopher Nolan]
118,tt0816692,165000000,621752480,Interstellar,Matthew McConaughey|Jessica Chastain|Anne Hath...,Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,Adventure|Drama|Science Fiction,Paramount Pictures|Legendary Pictures|Warner B...,11/5/2014,8.0,2014,456752480,"[Adventure, Drama, Science Fiction]",[Christopher Nolan]
125,tt2084970,14000000,233555708,The Imitation Game,Benedict Cumberbatch|Keira Knightley|Matthew G...,Morten Tyldum,The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,History|Drama|Thriller|War,Black Bear Pictures|Bristol Automotive,11/14/2014,8.0,2014,219555708,"[History, Drama, Thriller, War]",[Morten Tyldum]
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,6/9/2015,8.0,2015,678708609,"[Comedy, Animation, Family]",[Pete Docter]
34,tt3170832,6000000,35401758,Room,Brie Larson|Jacob Tremblay|Joan Allen|Sean Bri...,Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,Element Pictures|No Trace Camping|A24|Duperele...,10/16/2015,8.0,2015,29401758,"[Drama, Thriller]",[Lenny Abrahamson]
1183,tt0993846,100000000,392000694,The Wolf of Wall Street,Leonardo DiCaprio|Jonah Hill|Margot Robbie|Kyl...,Martin Scorsese,EARN. SPEND. PARTY.,A New York stockbroker refuses to cooperate in...,180,Crime|Drama|Comedy,Paramount Pictures|Appian Way|EMJAG Production...,12/25/2013,7.9,2013,292000694,"[Crime, Drama, Comedy]",[Martin Scorsese]
128,tt2267998,61000000,369330363,Gone Girl,Ben Affleck|Rosamund Pike|Carrie Coon|Neil Pat...,David Fincher,You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,Mystery|Thriller|Drama,Twentieth Century Fox Film Corporation|Regency...,10/1/2014,7.9,2014,308330363,"[Mystery, Thriller, Drama]",[David Fincher]
1191,tt2024544,20000000,187000000,12 Years a Slave,Chiwetel Ejiofor|Michael Fassbender|Lupita Nyo...,Steve McQueen,The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,Drama|History,Plan B Entertainment|Regency Enterprises|River...,10/18/2013,7.9,2013,167000000,"[Drama, History]",[Steve McQueen]
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,Marvel Studios|Moving Picture Company (MPC)|Bu...,7/30/2014,7.9,2014,603312399,"[Action, Science Fiction, Adventure]",[James Gunn]
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Peter Jackson,The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,Adventure|Fantasy|Action,WingNut Films|New Line Cinema,12/1/2003,7.9,2003,1024888979,"[Adventure, Fantasy, Action]",[Peter Jackson]


In [21]:
#Какие актеры чаще всего снимаются в одном фильме вместе?
from itertools import combinations
data_comb = data.copy()
data_comb['cast'] = data_comb.cast.str.split('|')

In [22]:
s = data_comb.cast.apply(lambda x: list(combinations(x, 2)))
l = []
for item in s:
    for i in item:
        i = sorted(i)
        k = i[0] + ' & ' + i [1]
        l.append(k)
type(Counter(l).most_common(1))

list